In [35]:
%pip install --upgrade --quiet langchain-exa 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

# Set the API key as an environment variable for LangChain
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

In [ ]:
!pip install mlflow

In [4]:
!python -m mlflow server --host 127.0.0.1 --port 5000

^C


In [5]:
import mlflow

In [14]:
from langchain_core.prompts import ChatPromptTemplate


In [27]:
from langchain_exa import ExaSearchResults

In [13]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [15]:
from langchain.chat_models import init_chat_model
model_name='gemini-2.0-flash'
model = init_chat_model(model_name, model_provider="google_genai")

In [16]:
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser

In [20]:
from langchain_core.tools import Tool

In [48]:
from mlflow.entities import SpanType

In [38]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("sentiment-detection2")

2025/09/21 16:47:31 INFO mlflow.tracking.fluent: Experiment with name 'sentiment-detection2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/688605823153491612', creation_time=1758453451985, experiment_id='688605823153491612', last_update_time=1758453451985, lifecycle_stage='active', name='sentiment-detection2', tags={}>

In [39]:
mlflow.langchain.autolog()

In [54]:
@mlflow.trace(span_type=SpanType.TOOL,  attributes={"FUNCTYPE": "No API"})
def get_stock_symbol(company_name):
    cmp = {"TCS": "TCS", "INFOSYS": "INFY", "RELIANCE": "RELIANCE", "WIPRO": "WIPRO", "ORACLE": "ORCL", "GOOGLE": "GOOG", "MICROSOFT": "MSFT", "APPLE": "AAPL", "AMAZON": "AMZN", "TESLA": "TSLA", "NVIDIA": "NVDA", "MICROSOFT": "MSFT", "APPLE": "AAPL", "AMAZON": "AMZN", "TESLA": "TSLA", "NVIDIA": "NVDA"}
    return cmp[company_name]
    # return {"company_name": company_name, "stock_code": cmp[company_name]}

In [55]:
@mlflow.trace(span_type="func",  attributes={"SearchAPI": "Exa Search"}, name="NewsFunc")
def get_news(stock_code):

    search_tools = ExaSearchResults(exa_api_key="83499386-0b01-4114-ad1e-b8f57e62a1d9")
    print(f"Top news for {stock_code}")
    op = search_tools._run(f"Get the latest news for the stock symbol{stock_code}", num_results=1, summary=True)
    return op.results[0].summary
    # return {"stock_code": stock_code, "news": op.results[0].summary}
    

In [56]:
get_stock_tool = Tool.from_function(
    name="search",
    description="useful when you need to get the top 3 news for the company",
    func=get_stock_symbol
)

In [57]:
news_tool = Tool.from_function(
    name="news",
    description="useful when you need to get the top 3 news for the company",
    func=get_news
)

In [45]:
prompt_template = """Get the sentiment of company {company_name}, stock code {stock_code} from the news summary {news}. Also, Extract named entities: people, places, other companies.
Provide a structured JSON with the following fields:
{{
"company_name": "",
"stock_code": "",
"newsdesc": "",
"sentiment": "Positive/Negative/Neutral",
"people_names": [],
"places_names": [],
"other_companies_referred": [],
"related_industries": [],
"market_implications": "",
"confidence_score": 0.0
}}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)


In [10]:
prompt = mlflow.genai.register_prompt(
    name = "stock-sentiment",
    template = prompt_template,
    tags = {
        "author": "Gaurav"
    }
)

2025/09/21 12:27:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: stock-sentiment, version 1


In [46]:
parser = JsonOutputParser()
# parser.invoke(test_res)


In [58]:
chain = RunnablePassthrough.assign( stock_code=get_stock_tool, news=news_tool) | prompt | model | parser
ans = chain.invoke({"company_name": "APPLE"})
print(ans)

2025/09/21 17:06:31 WARNING mlflow.tracing.export.mlflow_v3: Failed to log span to MLflow backend: INTERNAL_ERROR: Response: {'detail': 'REST OTLP span logging is not supported by FileStore'}
2025/09/21 17:06:31 WARNING mlflow.tracing.export.mlflow_v3: Failed to log span to MLflow backend: INTERNAL_ERROR: Response: {'detail': 'REST OTLP span logging is not supported by FileStore'}


Top news for APPLE


2025/09/21 17:06:36 WARNING mlflow.tracing.export.mlflow_v3: Failed to log span to MLflow backend: INTERNAL_ERROR: Response: {'detail': 'REST OTLP span logging is not supported by FileStore'}
2025/09/21 17:06:36 WARNING mlflow.tracing.export.mlflow_v3: Failed to log span to MLflow backend: INTERNAL_ERROR: Response: {'detail': 'REST OTLP span logging is not supported by FileStore'}
2025/09/21 17:06:36 WARNING mlflow.tracing.export.mlflow_v3: Failed to log span to MLflow backend: INTERNAL_ERROR: Response: {'detail': 'REST OTLP span logging is not supported by FileStore'}
2025/09/21 17:06:36 WARNING mlflow.tracing.export.mlflow_v3: Failed to log span to MLflow backend: INTERNAL_ERROR: Response: {'detail': 'REST OTLP span logging is not supported by FileStore'}
2025/09/21 17:06:37 WARNING mlflow.tracing.export.mlflow_v3: Failed to log span to MLflow backend: INTERNAL_ERROR: Response: {'detail': 'REST OTLP span logging is not supported by FileStore'}
2025/09/21 17:06:39 WARNING mlflow.traci

{'company_name': 'Apple', 'stock_code': 'AAPL', 'newsdesc': "Apple's iPhone 17 models are now available in Moscow stores, despite the company's official sales ban in Russia. This news appeared on benzinga.com on September 21, 2025.", 'sentiment': 'Neutral', 'people_names': [], 'places_names': ['Moscow', 'Russia'], 'other_companies_referred': [], 'related_industries': ['Technology', 'Consumer Electronics'], 'market_implications': "The availability of Apple products in Russia despite the sales ban could have several market implications. It could indicate a potential gray market for Apple products, impacting official sales channels. It could also be seen as a sign of continued demand for Apple products in Russia, even under sanctions. This could lead to potential re-evaluation of Apple's official stance on sales in Russia in the future. If apple is not involved, it may indicate some illegal activities which can lead to negative perception of the brand.", 'confidence_score': 0.8}


Trace(trace_id=tr-6890b7a3310f76c4ef5863792d5d1eee)

In [ ]:

mlflow.log_params({"model": "gemini-2.0-flash"})
mlflow.log_params({"parser": "json"})
mlflow.log_params({"prompt": prompt})
mlflow.log_params({"company_name": "GOOGLE"})
mlflow.log_metrics({"accuracy": 0.95})